In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import utils as u
import itertools

In [2]:
root_img_fold = "/home/doga/PycharmProjects/objectLocalization/dataset/wounds"
image_names = ['2021-09-01_photo1.jpg','2021-09-01_photo2_rl.jpg','2021-09-01_photo3_rr.jpg']
a_image_full_path = os.path.join(root_img_fold, image_names[0])
print(a_image_full_path)
img = mpimg.imread(a_image_full_path)
print("w({:}), h({:})".format(img.shape[0],img.shape[1]))

/home/doga/PycharmProjects/objectLocalization/dataset/wounds/2021-09-01_photo1.jpg
w(3024), h(4032)


In [3]:
hr = 6
wc = 6
rot_c=[int(hr/2),int(wc/2)]
rot_deg=45
a = np.arange(1,1+hr*wc).reshape((hr,wc))
b = np.arange(1,1+3*hr*wc).reshape((3,hr,wc))
print(a)
print(np.swapaxes(a,0,1))

[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]
 [25 26 27 28 29 30]
 [31 32 33 34 35 36]]
[[ 1  7 13 19 25 31]
 [ 2  8 14 20 26 32]
 [ 3  9 15 21 27 33]
 [ 4 10 16 22 28 34]
 [ 5 11 17 23 29 35]
 [ 6 12 18 24 30 36]]


In [4]:
rot_pixels, pix_vals_original, center_add = u.get_rotated_pixel_vals(wc, hr, rot_c=rot_c, rot_deg=rot_deg)
xyz = center_add+rot_pixels
print(xyz)

deg(45) = rad(0.7853981633974483)
[[-1.24264069  3.          1.        ]
 [-0.53553391  3.70710678  1.        ]
 [ 0.17157288  4.41421356  1.        ]
 [ 0.87867966  5.12132034  1.        ]
 [ 1.58578644  5.82842712  1.        ]
 [ 2.29289322  6.53553391  1.        ]
 [-0.53553391  2.29289322  1.        ]
 [ 0.17157288  3.          1.        ]
 [ 0.87867966  3.70710678  1.        ]
 [ 1.58578644  4.41421356  1.        ]
 [ 2.29289322  5.12132034  1.        ]
 [ 3.          5.82842712  1.        ]
 [ 0.17157288  1.58578644  1.        ]
 [ 0.87867966  2.29289322  1.        ]
 [ 1.58578644  3.          1.        ]
 [ 2.29289322  3.70710678  1.        ]
 [ 3.          4.41421356  1.        ]
 [ 3.70710678  5.12132034  1.        ]
 [ 0.87867966  0.87867966  1.        ]
 [ 1.58578644  1.58578644  1.        ]
 [ 2.29289322  2.29289322  1.        ]
 [ 3.          3.          1.        ]
 [ 3.70710678  3.70710678  1.        ]
 [ 4.41421356  4.41421356  1.        ]
 [ 1.58578644  0.17157288  1. 

###  assign_method == 'pick', nan_action == 'clip'

In [5]:
x = xyz[:,0].copy()
y = xyz[:,1].copy()
x1 = np.clip(np.round(x,0), 0, wc-1).astype('int')
y1 = np.clip(np.round(y,0), 0, hr-1).astype('int')
imTargetIdx = np.asarray(np.ravel_multi_index([x1, y1], (wc,hr)))
imSourceIdx = np.arange(0, len(imTargetIdx), dtype=int)
print(np.vstack((x,x1,y,y1,imSourceIdx,imTargetIdx)).T)

[[-1.24264069  0.          3.          3.          0.          3.        ]
 [-0.53553391  0.          3.70710678  4.          1.          4.        ]
 [ 0.17157288  0.          4.41421356  4.          2.          4.        ]
 [ 0.87867966  1.          5.12132034  5.          3.         11.        ]
 [ 1.58578644  2.          5.82842712  5.          4.         17.        ]
 [ 2.29289322  2.          6.53553391  5.          5.         17.        ]
 [-0.53553391  0.          2.29289322  2.          6.          2.        ]
 [ 0.17157288  0.          3.          3.          7.          3.        ]
 [ 0.87867966  1.          3.70710678  4.          8.         10.        ]
 [ 1.58578644  2.          4.41421356  4.          9.         16.        ]
 [ 2.29289322  2.          5.12132034  5.         10.         17.        ]
 [ 3.          3.          5.82842712  5.         11.         23.        ]
 [ 0.17157288  0.          1.58578644  2.         12.          2.        ]
 [ 0.87867966  1.        

###  assign_method == 'pick', nan_action in ['remove', 'available']:

In [6]:
x = xyz[:,0].copy()
y = xyz[:,1].copy()
iSoTa = [[s, int(np.ravel_multi_index([[int(i)],[int(j)]], (wc,hr)))] for s, (i, j) in enumerate(zip(x, y)) if (i<wc and i>=0 and j<hr and j>=0)]
iSoTa = np.asarray(iSoTa)
print(f"removed {len(x)-len(iSoTa)} pixels")
print(np.vstack((iSoTa[:,0],iSoTa[:,1])).T)

removed 5 pixels
[[ 2  4]
 [ 3  5]
 [ 4 11]
 [ 7  3]
 [ 8  3]
 [ 9 10]
 [10 17]
 [11 23]
 [12  1]
 [13  2]
 [14  9]
 [15 15]
 [16 22]
 [17 23]
 [18  0]
 [19  7]
 [20 14]
 [21 21]
 [22 21]
 [23 28]
 [24  6]
 [25 12]
 [26 19]
 [27 20]
 [28 27]
 [29 33]
 [31 18]
 [32 18]
 [33 25]
 [34 32]
 [35 33]]


###  assign_method == 'interpolate', nan_action == 'clip'
if clipping will happen then no removal will happen

In [7]:
x = xyz[:,0].copy()
y = xyz[:,1].copy()
xf, yf = np.floor(x).astype(int), np.floor(y).astype(int)
xc, yc = np.ceil(x).astype(int), np.ceil(y).astype(int)
xw, yw = x - xf, y - yf
Ax = np.clip(xf, 0, wc-1).astype('int')
Ay = np.clip(yf, 0, hr-1).astype('int')
A = np.ravel_multi_index([np.clip(xf, 0, wc-1).astype('int'),
                          np.clip(yf, 0, hr-1).astype('int')], (wc,hr)).astype(int)
B = np.ravel_multi_index([np.clip(xc, 0, wc-1).astype('int'),
                          np.clip(yf, 0, hr-1).astype('int')], (wc,hr)).astype(int)
C = np.ravel_multi_index([np.clip(xf, 0, wc-1).astype('int'),
                          np.clip(yc, 0, hr-1).astype('int')], (wc,hr)).astype(int)
D = np.ravel_multi_index([np.clip(xc, 0, wc-1).astype('int'),
                          np.clip(yc, 0, hr-1).astype('int')], (wc,hr)).astype(int)
#print(np.vstack((A,B,C,D)).T)
#print(a.ravel()[A])
#print(a[Ax,Ay])
a_final = np.zeros_like(a.ravel())
for i in range(4):
    xinds = np.clip(xf+i%2, 0, wc-1).astype('int')
    yinds = np.clip(yf+int(i/2), 0, hr-1).astype('int')
    kx1,kx2,ky1,ky2 = 1.0-i%2, (i%2-.5)*2, 1-int(i/2), (int(i/2)-.5)*2
    _wx =  kx1+kx2*xw
    _wy =  ky1+ky2*yw
    a_picked = a[xinds,yinds]
    #print(f'i{i}')
    #print(i%2,int(i/2),kx1,kx2, ky1,ky2 )
    #print(xinds.shape,yinds.shape,_wx.shape,_wy.shape,a_picked.shape)
    #print(a_picked)
    #print(a_picked*_wx*_wy)
    #print(fff2[i])
    a_final = a_final + a_picked*_wx*_wy
print(a_final.reshape(a.shape))
fff= [a[np.clip(xf+i%2, 0, wc-1).astype('int'),np.clip(yf+int(i/2), 0, hr-1).astype('int')]*((1.0-i%2)+((i%2-.5)*2)*xw)*((1-int(i/2))+((int(i/2)-.5)*2)*yw) for i in range(4)]
print(sum(fff).reshape(a.shape))

[[ 4.          4.70710678  6.44365081 11.27207794 15.51471863 19.75735931]
 [ 3.29289322  5.02943725  9.97918472 14.92893219 19.75735931 24.        ]
 [ 3.61522369  8.56497116 13.51471863 18.46446609 23.41421356 28.24264069]
 [ 7.1507576  12.10050506 17.05025253 22.         26.94974747 31.89949494]
 [10.6862915  15.63603897 20.58578644 25.53553391 30.48528137 34.70710678]
 [14.75735931 19.17157288 24.12132034 29.07106781 33.29289322 34.        ]]
[[ 4.          4.70710678  6.44365081 11.27207794 15.51471863 19.75735931]
 [ 3.29289322  5.02943725  9.97918472 14.92893219 19.75735931 24.        ]
 [ 3.61522369  8.56497116 13.51471863 18.46446609 23.41421356 28.24264069]
 [ 7.1507576  12.10050506 17.05025253 22.         26.94974747 31.89949494]
 [10.6862915  15.63603897 20.58578644 25.53553391 30.48528137 34.70710678]
 [14.75735931 19.17157288 24.12132034 29.07106781 33.29289322 34.        ]]


###  assign_method == 'interpolate', nan_action == 'remove'

In [8]:
x = xyz[:,0].copy()
y = xyz[:,1].copy()
xf, yf = np.floor(x).astype(int), np.floor(y).astype(int)
xw, yw = x - xf, y - yf
Ax,Ay = xf,yf
Bx,By = xc,yf
Cx,Cy = xf,yc
Dx,Dy = xc,yc
xinds = np.vstack((Ax,Bx,Cx,Dx)).T
xslct = ((xinds>=0).astype(int)*(xinds<wc).astype(int)).all(axis=1)
yinds = np.vstack((Ay,By,Cy,Dy)).T
yslct = ((yinds>=0).astype(int)*(yinds<hr).astype(int)).all(axis=1)
slct = ((xinds>=0).astype(int)*(xinds<wc).astype(int)*(yinds>=0).astype(int)*(yinds<hr).astype(int)).all(axis=1)
slct = ((xf >= 0).astype(int) * (xf<wc-1).astype(int) * (yf >= 0).astype(int) * (yf<hr-1).astype(int)).astype(bool)
print(np.where(slct))
a_final = np.zeros_like(a.ravel(), dtype=float)
ddd= [a[(xf+i%2)[slct],(yf+int(i/2))[slct]]*((1.0-i%2)+((i%2-.5)*2)*xw[slct])*((1-int(i/2))+((int(i/2)-.5)*2)*yw[slct])  for i in range(4)]
for i in range(4):
    xinds = xf+i%2
    yinds = yf+int(i/2)
    kx1,kx2,ky1,ky2 = 1.0-i%2, (i%2-.5)*2, 1-int(i/2), (int(i/2)-.5)*2
    _wx =  kx1+kx2*xw[slct]
    _wy =  ky1+ky2*yw[slct]
    a_picked = a[xinds[slct],yinds[slct]]
    print(f'i{i}')
    #print(i%2,int(i/2),kx1,kx2, ky1,ky2 )
    #print(xinds.shape,yinds.shape,_wx.shape,_wy.shape,a_picked.shape)
    print(ddd[i])
    print(a_picked*_wx*_wy)
    print(sum(ddd[i]-a_picked*_wx*_wy))
    a_final[slct] = a_final[slct]+a_picked*_wx*_wy
print(a_final.reshape(a.shape))
a_final3 = np.zeros_like(a.ravel())
a_final3[slct] = sum(ddd)
print(a_final3.reshape(a.shape))

(array([ 2,  7,  8,  9, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 31, 32, 33]),)
i0
[2.42640687e+00 3.31370850e+00 1.42135624e-01 2.66904756e+00
 6.86291501e-01 2.57359313e-01 4.14213562e+00 3.31370850e+00
 1.34730881e+01 1.47186258e-02 1.37258300e+00 7.50000000e+00
 2.20000000e+01 1.88730163e+00 9.95122676e+00 2.40202025e+00
 1.11522369e+00 8.28427125e+00 4.34924240e+00 1.64020203e+01
 1.57401154e+01 6.75144213e-01 6.30865787e+00]
[2.42640687e+00 3.31370850e+00 1.42135624e-01 2.66904756e+00
 6.86291501e-01 2.57359313e-01 4.14213562e+00 3.31370850e+00
 1.34730881e+01 1.47186258e-02 1.37258300e+00 7.50000000e+00
 2.20000000e+01 1.88730163e+00 9.95122676e+00 2.40202025e+00
 1.11522369e+00 8.28427125e+00 4.34924240e+00 1.64020203e+01
 1.57401154e+01 6.75144213e-01 6.30865787e+00]
0.0
i1
[ 1.1055557   1.71572875  2.57359313  5.83347776  0.56854249  5.59188309
  9.372583    1.88730163  0.          0.74621202  3.39696962  4.3492424
  0.          5.79898987  8.492

In [24]:
hr = 7
wc = 7
rot_c=[int(hr/2),int(wc/2)]
print(rot_c)
rot_deg=90
a = np.arange(1,1+hr*wc).reshape((hr,wc))
t_img = u.img_rotator()
t_img.fit(image=a, assign_method='pick', nan_action='clip', rot_c=rot_c, rot_deg=rot_deg)
t_img.apply_rotation()
print(t_img.image)
print(t_img.imRot)

[3, 3]
deg(90) = rad(1.5707963267948966)
[[ 1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19. 20. 21.]
 [22. 23. 24. 25. 26. 27. 28.]
 [29. 30. 31. 32. 33. 34. 35.]
 [36. 37. 38. 39. 40. 41. 42.]
 [43. 44. 45. 46. 47. 48. 49.]]
[[43. 36.  0. 22. 15.  8.  1.]
 [44. 37.  0. 23. 16. 10.  3.]
 [45. 38. 31. 24. 17.  0.  0.]
 [46. 39. 32. 25. 18. 11.  4.]
 [47. 40. 33. 26. 19. 12.  5.]
 [48. 41. 34. 27. 20. 13.  6.]
 [49. 42. 35. 28. 21. 14.  7.]]
